In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
from core.interpolator import Interpolator

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [ ]:
import os
from dotenv import load_dotenv
env_path = "templates/app_schema/.env"
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"Loaded environment variables from {env_path}")

In [4]:
application = Application(client, compiler)
interpolator = Interpolator(".")

In [5]:
application_description = """
Bot that does web search
""".strip()

In [6]:
application_description = "generate a straight forward hello world bot"

In [ ]:

my_bot = application.create_bot(application_description, bot_id="handler_tests")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Skipping Initial Description Refinement
Compiling TypeSpec...
Compiling Typescript Schema Definitions...
Compiling Drizzle...
Compiling Handler Tests...


: 

: 

In [17]:
interpolator.bake(my_bot, "../../bot_result") # next to repo root

In [ ]:
my_bot.__dict__

{'refined_description': RefineOut(refined_description='generate a straight forward hello world bot', error_output=None),
 'typespec': TypespecOut(reasoning='For a straightforward "hello world" bot, I need to think about the basic interactions:\n\n1. The core functionality should be greeting the user, which could be triggered by various user inputs like "hello", "hi", etc.\n2. We might want to allow the user to introduce themselves, so the bot can greet them by name.\n3. A simple help function could explain what the bot can do.\n\nThese would map to the following functions:\n- greetUser: Simple greeting without requiring user details\n- greetWithName: Personalized greeting using the user\'s name\n- getHelp: Provides information about the bot\'s capabilities\n\nEach function needs appropriate scenarios, and the models need to be simple but cover the necessary information.', typespec_definitions='model GreetingOptions {\n    greeting: string;\n}\n\nmodel NamedGreetingOptions {\n    name: 